In [1]:
import os
from bs4 import BeautifulSoup
import json

def extract_gt_data_from_html(html_file_path):
    """HTMLファイルからimg srcのパスとgt textを抽出する"""
    with open(html_file_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
    
    results = []
    
    # 各sampleを処理
    samples = soup.find_all('div', class_='sample')
    
    for sample in samples:
        # img srcを取得
        img_tag = sample.find('img')
        if img_tag and img_tag.get('src'):
            img_path = img_tag['src']
            
            # class="text-content gt"のテキストを取得
            gt_div = sample.find('div', class_='text-content gt')
            if gt_div:
                gt_text = gt_div.get_text(strip=True)
                
                results.append({
                    "path": img_path,
                    "gt": gt_text
                })
    
    return results


In [4]:

# datasets/html_visualization_appen/のHTMLファイルを処理
# html_dir = "../datasets/html_visualization_appen"
html_dir = "../datasets/html_visualization_datatang"
all_gt_data = []

# for i in range(4):  # batch_0 から batch_3 まで
for i in range(16):  # batch_0 から batch_15 まで
    html_file = f"ocr_results_batch_{i}.html"
    html_path = os.path.join(html_dir, html_file)
    
    if os.path.exists(html_path):
        print(f"Processing {html_file}...")
        batch_data = extract_gt_data_from_html(html_path)
        all_gt_data.extend(batch_data)
        print(f"Extracted {len(batch_data)} entries from {html_file}")
    else:
        print(f"File not found: {html_path}")

print(f"\nTotal entries extracted: {len(all_gt_data)}")

# 結果を表示（最初の数件）
print("\nFirst 3 entries:")
for i, entry in enumerate(all_gt_data[:3]):
    print(f"{i+1}. Path: {entry['path']}")
    print(f"   GT: {entry['gt'][:100]}...")  # 最初の100文字のみ表示
    print()

# JSONファイルとして保存
output_file = html_dir + "/gt_dataset.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_gt_data, f, ensure_ascii=False, indent=2)

print(f"Dataset saved to: {output_file}")


Processing ocr_results_batch_0.html...
Extracted 512 entries from ocr_results_batch_0.html
Processing ocr_results_batch_1.html...
Extracted 512 entries from ocr_results_batch_1.html
Processing ocr_results_batch_2.html...
Extracted 512 entries from ocr_results_batch_2.html
Processing ocr_results_batch_3.html...
Extracted 512 entries from ocr_results_batch_3.html
Processing ocr_results_batch_4.html...
Extracted 512 entries from ocr_results_batch_4.html
Processing ocr_results_batch_5.html...
Extracted 512 entries from ocr_results_batch_5.html
Processing ocr_results_batch_6.html...
Extracted 512 entries from ocr_results_batch_6.html
Processing ocr_results_batch_7.html...
Extracted 512 entries from ocr_results_batch_7.html
Processing ocr_results_batch_8.html...
Extracted 512 entries from ocr_results_batch_8.html
Processing ocr_results_batch_9.html...
Extracted 512 entries from ocr_results_batch_9.html
Processing ocr_results_batch_10.html...
Extracted 512 entries from ocr_results_batch_10.ht

In [4]:
# 一旦出力したgt_dataset.jsonを取得して、その中身を繋げる。
# その後、電話番号の下4桁を抽出して、その直後に改行を入れる。

import json
import re

# datasets/html_visualization_appen/gt_dataset.json を読み込み
output_file = "../datasets/html_visualization_appen/gt_dataset.json"
html_dir = "../datasets/html_visualization_appen"

with open(output_file, 'r', encoding='utf-8') as f:
    gt_data = json.load(f)

# 全てのgtテキストを繋げる
combined_text = ""
for entry in gt_data:
    combined_text += entry['gt']

print(f"Combined text length: {len(combined_text)}")
print(f"First 200 characters: {combined_text[:200]}")

# 電話番号パターンを検索して、下4桁の直後に改行を挿入


# 電話番号パターン（例：03-5945-6185, 070-7440-7013など）
# ハイフンで区切られた最後の4桁の数字の後に改行を挿入
phone_pattern = r'(\d{2,4}-\d{3,4}-\d{4})'

def add_newline_after_phone(text):
    # 電話番号パターンを見つけて、その後に改行を追加
    result = re.sub(phone_pattern, r'\1\n', text)
    return result

# 電話番号の後に改行を挿入
processed_text = add_newline_after_phone(combined_text)

print(f"\nProcessed text length: {len(processed_text)}")
print("First 300 characters of processed text:")
print(processed_text[:300])

# 処理済みテキストを保存
processed_output_file = html_dir + "/combined_gt_text.txt"
with open(processed_output_file, 'w', encoding='utf-8') as f:
    f.write(processed_text)

print(f"\nProcessed text saved to: {processed_output_file}")
 

Combined text length: 9738
First 200 characters: 外山茉梨愛知県豊川市赤坂町東裏林コーポ952号室昭和52年07月13日03-5945-6185奥原阿満里山形県酒田市こがね町一丁目13-54-18昭和32年03月12日249013165宮澤得雄宮城県伊具郡丸森町字福山山本レジデンス北棟343号室1966年10月23日070-7440-7013瀬戸口晃丈郡山市熱海町安子島追分54番地-2号1977.06.27090-9596-3504城山人也福岡県

Processed text length: 9891
First 300 characters of processed text:
外山茉梨愛知県豊川市赤坂町東裏林コーポ952号室昭和52年07月13日03-5945-6185
奥原阿満里山形県酒田市こがね町一丁目13-54-18昭和32年03月12日249013165宮澤得雄宮城県伊具郡丸森町字福山山本レジデンス北棟343号室1966年10月23日070-7440-7013
瀬戸口晃丈郡山市熱海町安子島追分54番地-2号1977.06.27090-9596-3504
城山人也福岡県北九州市八幡東区中メ田一丁目前田グランリ階-8号室H9.03.1206-3872-7651
畑井唯宮城県加美郡加美町字原三本松14-67-21958/04/15090-7262-9668
蜂谷真

Processed text saved to: ../datasets/html_visualization_appen/combined_gt_text.txt
